In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/code/reversible/reversible2/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/explaining/reversible//')
%cd /home/schirrmr/


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
from matplotlib import rcParams, cycler
import seaborn
seaborn.set_style('darkgrid')

from reversible.sliced import sliced_from_samples

from numpy.random import RandomState

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
from reversible.plot import create_bw_image
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np
from reversible.revnet import ResidualBlock, invert, SubsampleSplitter, ViewAs, ReversibleBlockOld
from spectral_norm import spectral_norm
from conv_spectral_norm import conv_spectral_norm

def display_text(text, fontsize=18):
    fig = plt.figure(figsize=(12,0.1))
    plt.title(text, fontsize=fontsize)
    plt.axis('off')
    display(fig)
    plt.close(fig)

In [ ]:
from braindecode.datasets.bbci import BBCIDataset
from braindecode.mne_ext.signalproc import mne_apply
# we loaded all sensors to always get same cleaning results independent of sensor selection
# There is an inbuilt heuristic that tries to use only EEG channels and that definitely
# works for datasets in our paper
#train_loader = BBCIDataset('/data/schirrmr/schirrmr/HGD-public/reduced/train/13.mat')
#test_loader = BBCIDataset('/data/schirrmr/schirrmr/HGD-public/reduced/test/13.mat')
start_cnt = BBCIDataset('/data/schirrmr/schirrmr/HGD-public/reduced/train/4.mat',).load()
start_cnt = start_cnt.drop_channels(['STI 014'])
def car(a):
    return a - np.mean(a, keepdims=True, axis=0)

start_cnt = mne_apply(
    car, start_cnt)

start_cnt = start_cnt.reorder_channels(['C3', 'C4'])

In [ ]:
from collections import OrderedDict
from braindecode.datautil.trial_segment import create_signal_target_from_raw_mne

marker_def = OrderedDict([('Right Hand', [1]), ('Left Hand', [2],),
                         ('Rest', [3]), ('Feet', [4])])
ival = [500,1500]
from braindecode.mne_ext.signalproc import mne_apply, resample_cnt
from braindecode.datautil.signalproc import exponential_running_standardize, bandpass_cnt

log.info("Resampling train...")
cnt = resample_cnt(start_cnt, 250.0)
log.info("Standardizing train...")
cnt = mne_apply(lambda a: exponential_running_standardize(a.T ,factor_new=1e-3, init_block_size=1000, eps=1e-4).T,
                     cnt)
cnt = resample_cnt(cnt, 32.0)
cnt = resample_cnt(cnt, 64.0)
#cnt = mne_apply(
#    lambda a: bandpass_cnt(a, 0, 2, cnt.info['sfreq'],
#                           filt_order=10,
#                           axis=1), cnt)

train_set = create_signal_target_from_raw_mne(cnt, marker_def, ival)
cnt_bandpassed =  mne_apply(
    lambda a: bandpass_cnt(a, 8, 13, cnt.info['sfreq'],
                           filt_order=10,
                           axis=1), cnt)
alpha_set = create_signal_target_from_raw_mne(cnt_bandpassed, marker_def, ival)
x_alpha_right = alpha_set.X[alpha_set.y == 0]

x_alpha_rest = alpha_set.X[alpha_set.y == 2]

alpha_a = np_to_var(x_alpha_right[:160,0:1,:,None], dtype=np.float32)

alpha_b = np_to_var(x_alpha_rest[:160,0:1,:,None], dtype=np.float32)
inputs_alpha = [alpha_a, alpha_b]


In [ ]:
i_class = 0
a_in = inputs_alpha[i_class][0]
plt.plot(var_to_np(a_in).squeeze())

## single signal

In [ ]:
from reversible.util import set_random_seeds

set_random_seeds(3, True)
templates = th.zeros((2,inputs_alpha[i_class].shape[2] // 2), requires_grad=True)

templates.data[0] = th.linspace(0,63,64)[::2]
templates.data[1] = th.linspace(0,63,64)[1::2]

templates.data[0] = th.randn(32) * 0.5
templates.data[1] = th.randn(32) * 0.5

coef_a = th.ones(1, requires_grad=True)
coef_b = th.ones(1, requires_grad=True)

a_b = templates * th.cat((coef_a, coef_b)).unsqueeze(1)
a_b.unsqueeze(0).unsqueeze(-1).shape
inverted = invert(SubsampleSplitter((2,1), chunk_chans_first=False), a_b.unsqueeze(0).unsqueeze(-1)).squeeze()

fig = plt.figure(figsize=(12,2))
plt.plot(var_to_np(inverted).squeeze())
plt.plot(var_to_np(a_in).squeeze())
display(fig)
plt.close(fig)

In [ ]:
optimizer = th.optim.Adam([templates], lr=1e-2)#, coef_a, coef_b
n_epochs = 1000
for i_epoch in range(n_epochs):
    a_b = templates * th.cat((coef_a, coef_b)).unsqueeze(1)
    a_b.unsqueeze(0).unsqueeze(-1).shape
    inverted = invert(SubsampleSplitter((2,1), chunk_chans_first=False), a_b.unsqueeze(0).unsqueeze(-1)).squeeze()
    loss = th.sqrt(th.sum((inverted - a_in.squeeze()) ** 2))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i_epoch % (n_epochs // 10) == 0:
        print("Loss: {:.4f}".format(loss.item()))
        fig = plt.figure(figsize=(12,2))
        plt.plot(var_to_np(inverted).squeeze())
        plt.plot(var_to_np(a_in).squeeze())
        display(fig)
        plt.close(fig)

## all signals

In [ ]:
from reversible.util import set_random_seeds

set_random_seeds(3, True)
templates = th.zeros((2,inputs_alpha[i_class].shape[2] // 2), requires_grad=True)


templates.data[0] = th.randn(32) * 0.5
templates.data[1] = th.randn(32) * 0.5

coef_a = th.ones(len(inputs_alpha[i_class]), requires_grad=True)
coef_b = th.ones(len(inputs_alpha[i_class]), requires_grad=True)
a_b = templates.unsqueeze(0) * th.stack((coef_a, coef_b), dim=1).unsqueeze(2)
inverted = invert(SubsampleSplitter((2,1), chunk_chans_first=False), a_b.unsqueeze(-1))

fig = plt.figure(figsize=(12,2))
plt.plot(var_to_np(inverted[0]).squeeze())
plt.plot(var_to_np(inputs_alpha[i_class][0]).squeeze())
display(fig)
plt.close(fig)

In [ ]:
optimizer = th.optim.Adam([templates, coef_a, coef_b], lr=1e-2)#, coef_a, coef_b
n_epochs = 10000
for i_epoch in range(n_epochs):
    a_b = templates.unsqueeze(0) * th.stack((coef_a, coef_b), dim=1).unsqueeze(2)
    inverted = invert(SubsampleSplitter((2,1), chunk_chans_first=False), a_b.unsqueeze(-1))
    loss = th.mean(th.sqrt(th.sum((inverted - inputs_alpha[i_class]) ** 2, dim=2)).squeeze())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i_epoch % (n_epochs // 10) == 0:
        print("Loss: {:.4f}".format(loss.item()))
        fig = plt.figure(figsize=(12,2))
        plt.plot(var_to_np(inverted[0]).squeeze())
        plt.plot(var_to_np(inputs_alpha[i_class][0]).squeeze())
        display(fig)
        plt.close(fig)

In [ ]:
fig = plt.figure(figsize=(10,3))
plt.plot(var_to_np(inverted[0]).squeeze(), color=seaborn.color_palette()[0])
plt.plot(var_to_np(inputs_alpha[i_class][0]).squeeze(),
        color=seaborn.color_palette()[1])
plt.legend(["Real", "Fake"])
display(fig)

plt.close(fig)

In [ ]:
a_b = templates.unsqueeze(0) * th.stack((coef_a, coef_b), dim=1).unsqueeze(2)
inverted = invert(SubsampleSplitter((2,1), chunk_chans_first=False), a_b.unsqueeze(-1))

fig, axes = plt.subplots(5,8, figsize=(18,12), sharex=True, sharey=True)
for i_in, ax in enumerate(axes.flatten()):
    ax.plot(var_to_np(inverted[i_in]).squeeze(), color=seaborn.color_palette()[0],)
    ax.plot(var_to_np(inputs_alpha[i_class][i_in]).squeeze(), color='black')
display(fig)
plt.close(fig)


In [ ]:
plt.figure(figsize=(10,3))
plt.plot(var_to_np(templates).T)
plt.title("Templates")

In [ ]:
a_b = templates.unsqueeze(0)
inverted = invert(SubsampleSplitter((2,1), chunk_chans_first=False), a_b.unsqueeze(-1))
fig = plt.figure(figsize=(10,3))
plt.plot(var_to_np(inverted[0]).squeeze())
plt.title("Full Template")

In [ ]:
plt.plot(np.fft.rfftfreq(len(inverted.squeeze()),d=1/64.0),
    np.abs(np.fft.rfft(var_to_np(inverted).squeeze())))

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(var_to_np(coef_a), var_to_np(coef_b), ls='', marker='o')
plt.xlim(-2,2)
plt.ylim(-2,2)

### two full templates

In [ ]:
# two full templates
# half templates with invertible network after

In [ ]:
from reversible.util import set_random_seeds

set_random_seeds(3, True)
templates = th.zeros((2,inputs_alpha[i_class].shape[2] ), requires_grad=True)


templates.data[0] = th.randn(64) * 0.5
templates.data[1] = th.randn(64) * 0.5

coef_a = th.ones(len(inputs_alpha[i_class]), requires_grad=True)
coef_b = th.ones(len(inputs_alpha[i_class]), requires_grad=True)
a_b = templates.unsqueeze(0) * th.stack((coef_a, coef_b), dim=1).unsqueeze(2)
inverted = (a_b[:,0] + a_b[:,1]).unsqueeze(1).unsqueeze(-1)


fig = plt.figure(figsize=(12,2))
plt.plot(var_to_np(inverted[0]).squeeze())
plt.plot(var_to_np(inputs_alpha[i_class][0]).squeeze())
display(fig)
plt.close(fig)

In [ ]:
optimizer = th.optim.Adam([templates, coef_a, coef_b], lr=1e-2)#, coef_a, coef_b
n_epochs = 10000
for i_epoch in range(n_epochs):
    a_b = templates.unsqueeze(0) * th.stack((coef_a, coef_b), dim=1).unsqueeze(2)
    inverted = (a_b[:,0] + a_b[:,1]).unsqueeze(1).unsqueeze(-1)
    loss = th.mean(th.sqrt(th.sum((inverted - inputs_alpha[i_class]) ** 2, dim=2)).squeeze())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i_epoch % (n_epochs // 10) == 0:
        print("Loss: {:.4f}".format(loss.item()))
        fig = plt.figure(figsize=(12,2))
        plt.plot(var_to_np(inverted[0]).squeeze())
        plt.plot(var_to_np(inputs_alpha[i_class][0]).squeeze())
        display(fig)
        plt.close(fig)

In [ ]:
fig = plt.figure(figsize=(10,3))
plt.plot(var_to_np(inverted[0]).squeeze(), color=seaborn.color_palette()[0])
plt.plot(var_to_np(inputs_alpha[i_class][0]).squeeze(),
        color=seaborn.color_palette()[1])
plt.legend(["Real", "Fake"])
display(fig)

plt.close(fig)

In [ ]:
plt.figure(figsize=(10,3))
plt.plot(var_to_np(templates).T)
plt.title("Templates")

In [ ]:
plt.figure(figsize=(10,3))
plt.plot(np.fft.rfftfreq(64, 1/64.0), np.abs(np.fft.rfft(var_to_np(templates))).T)
plt.title("Spectrum of templates")

### half templates with invertible network

In [ ]:
def rev_block(n_c, n_i_c):
     return ReversibleBlockOld(
        nn.Sequential(
            nn.Conv2d(n_c // 2, n_i_c, (1,1)),
            nn.ReLU(),
            nn.Conv2d(n_i_c, n_c // 2, (1,1))),
        nn.Sequential(
            nn.Conv2d(n_c // 2, n_i_c, (1,1)),
            nn.ReLU(),
            nn.Conv2d(n_i_c, n_c // 2, (1,1))),
    )

In [ ]:
from reversible.util import set_random_seeds

set_random_seeds(3, True)
templates = th.zeros((2,inputs_alpha[i_class].shape[2] // 2), requires_grad=True)


templates.data[0] = th.randn(32) * 0.5
templates.data[1] = th.randn(32) * 0.5

coef_a = th.ones(len(inputs_alpha[i_class]), requires_grad=True)
coef_b = th.ones(len(inputs_alpha[i_class]), requires_grad=True)
a_b = templates.unsqueeze(0) * th.stack((coef_a, coef_b), dim=1).unsqueeze(2)

feature_model = nn.Sequential(
    SubsampleSplitter((2,1), chunk_chans_first=False),
    rev_block(2,32),
    rev_block(2,32),
)

inverted = invert(feature_model, a_b.unsqueeze(-1))

fig = plt.figure(figsize=(12,2))
plt.plot(var_to_np(inputs_alpha[i_class][0]).squeeze())
plt.plot(var_to_np(inverted[0]).squeeze())
display(fig)
plt.close(fig)

In [ ]:
optimizer = th.optim.Adam([templates, coef_a, coef_b] + list(feature_model.parameters()), lr=1e-3)
n_epochs = 10000
for i_epoch in range(n_epochs):
    a_b = templates.unsqueeze(0) * th.stack((coef_a, coef_b), dim=1).unsqueeze(2)
    inverted = invert(feature_model, a_b.unsqueeze(-1))
    loss = th.mean(th.sqrt(th.sum((inverted - inputs_alpha[i_class]) ** 2, dim=2)).squeeze())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i_epoch % (n_epochs // 10) == 0:
        print("Loss: {:.4f}".format(loss.item()))
        fig = plt.figure(figsize=(12,2))
        plt.plot(var_to_np(inputs_alpha[i_class][0]).squeeze())
        plt.plot(var_to_np(inverted[0]).squeeze())
        display(fig)
        plt.close(fig)

In [ ]:
plt.figure(figsize=(10,3))
plt.plot(var_to_np(templates).T)
plt.title("Templates")

In [ ]:
a_b = templates.unsqueeze(0)
inverted = invert(SubsampleSplitter((2,1), chunk_chans_first=False), a_b.unsqueeze(-1))
fig = plt.figure(figsize=(10,3))
plt.plot(var_to_np(inverted[0]).squeeze())
plt.title("Full Template")


### with bigger network

In [ ]:
from reversible.util import set_random_seeds

set_random_seeds(3, True)
templates = th.zeros((2,inputs_alpha[i_class].shape[2] // 2), requires_grad=True)


templates.data[0] = th.randn(32) * 0.5
templates.data[1] = th.randn(32) * 0.5

coef_a = th.ones(len(inputs_alpha[i_class]), requires_grad=True)
coef_b = th.ones(len(inputs_alpha[i_class]), requires_grad=True)
a_b = templates.unsqueeze(0) * th.stack((coef_a, coef_b), dim=1).unsqueeze(2)

feature_model = nn.Sequential(
    SubsampleSplitter((2,1), chunk_chans_first=False),
    rev_block(2,32),
    rev_block(2,32),
    rev_block(2,32),
    rev_block(2,32),
    rev_block(2,32),
    rev_block(2,32),
)

inverted = invert(feature_model, a_b.unsqueeze(-1))

fig = plt.figure(figsize=(12,2))
plt.plot(var_to_np(inputs_alpha[i_class][0]).squeeze())
plt.plot(var_to_np(inverted[0]).squeeze())
display(fig)
plt.close(fig)

In [ ]:
optimizer = th.optim.Adam([templates, coef_a, coef_b] + list(feature_model.parameters()), lr=1e-3)
n_epochs = 10000
for i_epoch in range(n_epochs):
    a_b = templates.unsqueeze(0) * th.stack((coef_a, coef_b), dim=1).unsqueeze(2)
    inverted = invert(feature_model, a_b.unsqueeze(-1))
    loss = th.mean(th.sqrt(th.sum((inverted - inputs_alpha[i_class]) ** 2, dim=2)).squeeze())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i_epoch % (n_epochs // 10) == 0:
        print("Loss: {:.4f}".format(loss.item()))
        fig = plt.figure(figsize=(12,2))
        plt.plot(var_to_np(inputs_alpha[i_class][0]).squeeze())
        plt.plot(var_to_np(inverted[0]).squeeze())
        display(fig)
        plt.close(fig)

### with fixed templates

In [ ]:
from reversible.util import set_random_seeds

set_random_seeds(3, True)
templates = th.zeros((2,inputs_alpha[i_class].shape[2] // 2), requires_grad=True)


templates.data[0] = th.ones(32) * 0.5
templates.data[1] = th.ones(32) * 0.5

coef_a = th.ones(len(inputs_alpha[i_class]), requires_grad=True)
coef_b = th.ones(len(inputs_alpha[i_class]), requires_grad=True)
a_b = templates.unsqueeze(0) * th.stack((coef_a, coef_b), dim=1).unsqueeze(2)

feature_model = nn.Sequential(
    SubsampleSplitter((2,1), chunk_chans_first=False),
    rev_block(2,32),
    rev_block(2,32),
)

inverted = invert(feature_model, a_b.unsqueeze(-1))

fig = plt.figure(figsize=(12,2))
plt.plot(var_to_np(inputs_alpha[i_class][0]).squeeze())
plt.plot(var_to_np(inverted[0]).squeeze())
display(fig)
plt.close(fig)

In [ ]:
optimizer = th.optim.Adam([coef_a, coef_b] + list(feature_model.parameters()), lr=1e-3)
n_epochs = 10000
for i_epoch in range(n_epochs):
    a_b = templates.unsqueeze(0) * th.stack((coef_a, coef_b), dim=1).unsqueeze(2)
    inverted = invert(feature_model, a_b.unsqueeze(-1))
    loss = th.mean(th.sqrt(th.sum((inverted - inputs_alpha[i_class]) ** 2, dim=2)).squeeze())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i_epoch % (n_epochs // 10) == 0:
        print("Loss: {:.4f}".format(loss.item()))
        fig = plt.figure(figsize=(12,2))
        plt.plot(var_to_np(inputs_alpha[i_class][0]).squeeze())
        plt.plot(var_to_np(inverted[0]).squeeze())
        display(fig)
        plt.close(fig)

In [ ]:
64/8 .. # we need 8 samples

In [ ]:
# on the "correct" level (upper bound) (so 8)
# for this sample, model it, so optimize coefficients at this level
